In [100]:
import pandas as pd
import numpy as np
import os

import time
from alibi_detect.od import SpectralResidual

In [101]:
def get_data():
    data_path = '../../../../data/train_data/host'
    dfs = {}
    for file in os.listdir(data_path):
        print('Reading ' + file)
        dfs[file[:-4]] = pd.read_csv(data_path+'/'+file) 
    return dfs

def load_tiago_test(test):
    data_path = '../../../../data/tiago_tests/test'
    df = pd.read_csv(data_path+str(test)+'/kpis.csv')
    return df

def load_test():
    data_path = '../../../../data/test_data/host'
    df = pd.DataFrame(['item_id','name','bomc_id','timestamp','value','cmdb_id'])
    for file in os.listdir(data_path):
        df = pd.concatenate(df, pd.read_csv(data_path+'/'+file)) 
    return dfs

In [102]:
def get_past(df, curr_time, window_size):
    step_size = 1000*60
    df_new = pd.DataFrame(columns=['timestamp', 'value'])

    for time in [curr_time - step_size*x for x in range(window_size)][::-1]:
        if len(df[df.timestamp == time]) == 0:
            plug_time = np.max(df[df.timestamp < time]['timestamp'])
            row = {}
            row['timestamp'] = time
            if len(df.loc[df.timestamp==plug_time, 'value']) == 0:
                row['value'] = 0
            else: 
                row['value'] = df.loc[df.timestamp==plug_time, 'value'].values[0]
            df_new = df_new.append(row, ignore_index=True)
        else:
            row = {}
            row['timestamp'] = time
            row['value'] = df.loc[df.timestamp==time, 'value'].values[0]
            df_new = df_new.append(row, ignore_index=True)
    return df_new

In [ ]:
def get_key_thresh(host):
    df_thresh = pd.read_csv('thresh.csv')
    df_thresh = df_thresh[df_thresh.host==host]

    thresh = df_thresh[['name', 'thresh']].set_index('name')
    thresh = thresh.to_dict()['thresh']
    return thresh

def find_anoms(hosts, df):
    start = time.time()
    anoms = []
    for host in hosts:
        start_host = time.time()
        print('*'*60)
        print('Checking ', host)
        thresholds = get_key_thresh(host)
        for kpi in thresholds:
            thresh = thresholds[kpi]
            print('Threshold ', thresh, kpi)
            od = SpectralResidual(
                threshold=thresh,
                window_amp=30,
                window_local=30,
                n_est_points=5,
                n_grad_points=5
            )
            df_hk = df[df.cmdb_id == host][df.name == kpi]
            
            if len(df_hk) == 0:
                print('No valid data to use, skipping')
                continue
                
            data = get_past(df_hk, np.max(df_hk.timestamp.unique()), 30)['value']
            if thresh == -1:
                if sum(data[-2:]) > 0:
                    print("Non-Zero Anomaly!")
                    anoms.append((host, kpi))
            elif np.isnan(thresh):
                print('NaN, Skipping!')
            else:
                outliers = od.predict(data.values)['data']
                if np.sum(outliers['is_outlier'][-10:-5]) > 0:
                    print(outliers['is_outlier'])
                    print("ST Threshold Anomaly!")
                    anoms.append((host,kpi))
        print(host, ' completed in ', time.time() - start_host)
    print('Completed detection of ', len(hosts), 'hosts in ', time.time() - start, 'seconds')
    return anoms


find_anoms(load_tiago_test(4).cmdb_id.unique(), load_tiago_test(4))

************************************************************
Checking  db_011
Threshold  0.857604463736166 CPU_free_pct
[1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
ST Threshold Anomaly!
Threshold  0.8565888283685505 MEM_real_util
[1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
ST Threshold Anomaly!
Threshold  0.7800049643982482 tnsping_result_time


<ipython-input-106-934139de14b0>:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_hk = df[df.cmdb_id == host][df.name == kpi]
<ipython-input-106-934139de14b0>:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_hk = df[df.cmdb_id == host][df.name == kpi]
<ipython-input-106-934139de14b0>:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_hk = df[df.cmdb_id == host][df.name == kpi]
<ipython-input-106-934139de14b0>:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_hk = df[df.cmdb_id == host][df.name == kpi]


Threshold  0.9736504655586444 CPU_Used_Pct
Threshold  0.8558544935510687 MEM_Used
[1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
ST Threshold Anomaly!
Threshold  nan MEM_Total
NaN, Skipping!
Threshold  0.8503225686445326 MEM_Used_Pct


<ipython-input-106-934139de14b0>:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_hk = df[df.cmdb_id == host][df.name == kpi]
<ipython-input-106-934139de14b0>:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_hk = df[df.cmdb_id == host][df.name == kpi]
<ipython-input-106-934139de14b0>:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_hk = df[df.cmdb_id == host][df.name == kpi]


[1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
ST Threshold Anomaly!
Threshold  0.8576184941963902 Used_Tbs_Size
Threshold  nan SEQ_Used_Pct
NaN, Skipping!
Threshold  nan Asm_Free_Tb
NaN, Skipping!

...

<ipython-input-106-934139de14b0>:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_hk = df[df.cmdb_id == host][df.name == kpi]
<ipython-input-106-934139de14b0>:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_hk = df[df.cmdb_id == host][df.name == kpi]
<ipython-input-106-934139de14b0>:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_hk = df[df.cmdb_id == host][df.name == kpi]
<ipython-input-106-934139de14b0>:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_hk = df[df.cmdb_id == host][df.name == kpi]



Threshold  -1.0 evicted_keys
Threshold  2.8407200534991164 used_cpu_user
Threshold  0.8576776422732205 used_memory
[1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
ST Threshold Anomaly!
Threshold  nan maxmemory
NaN, Skipping!

<ipython-input-106-934139de14b0>:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_hk = df[df.cmdb_id == host][df.name == kpi]
<ipython-input-106-934139de14b0>:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_hk = df[df.cmdb_id == host][df.name == kpi]
<ipython-input-106-934139de14b0>:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_hk = df[df.cmdb_id == host][df.name == kpi]
<ipython-input-106-934139de14b0>:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_hk = df[df.cmdb_id == host][df.name == kpi]



Threshold  2.7640504192731887 used_cpu_sys
redis_008  completed in  1.2522168159484863
************************************************************
Checking  redis_002
Threshold  0.8576556130119274 used_memory_rss
[1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
ST Threshold Anomaly!
Threshold  -1.0 rejected_connections
Threshold  0.8573728126632113 keyspace_misses


<ipython-input-106-934139de14b0>:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_hk = df[df.cmdb_id == host][df.name == kpi]
<ipython-input-106-934139de14b0>:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_hk = df[df.cmdb_id == host][df.name == kpi]
<ipython-input-106-934139de14b0>:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_hk = df[df.cmdb_id == host][df.name == kpi]
<ipython-input-106-934139de14b0>:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_hk = df[df.cmdb_id == host][df.name == kpi]


In [90]:
import seaborn as sns

df = load_test(2)
df = df[df.cmdb_id=='os_018'][df.name=='Buffers_used']
sns.lineplot(x='timestamp', y='value',data=df)

TypeError: load_test() takes 0 positional arguments but 1 was given